---
title:  "Neuralnet "
excerpt: "Basic Neural Net using numpy,tensor-flow,keras"

categories:
  - Deep-Learning
tags:
  - DL
  - Neuralnet Using Numpy
  - 밑바닥부터 시작하는 딥러닝
  - 딥러닝
last_modified_at: 2020-02-16T08:06:00-05:00
---

##### MNIST 데이터를 활용하여, 기본 뉴럴넷을 구성해보자
> minst 데이터는 기본 keras에 있는 걸 활용한다.

In [5]:
import keras
keras.__version__

'2.2.4'

In [7]:
from keras.datasets import mnist
(train_images,train_labels),(test_images, test_labels) = mnist.load_data()

In [9]:
print("train_images.shape",train_images.shape)
print("train_labels.shape",train_labels.shape)
print("test_images.shape",test_images.shape)
print("test_labels",test_labels.shape)

train_images.shape (60000, 28, 28)
train_labels.shape (60000,)
test_images.shape (10000, 28, 28)
test_labels (10000,)
